# Setup

In [13]:
import os
import requests
import pandas as pd
import numpy as np
import matplotlib as mpl
mpl.rc('axes', labelsize=30)
mpl.rc('font', size=30)
mpl.rc('xtick', labelsize=22)
mpl.rc('ytick', labelsize=22)
from prophet import Prophet
from sklearn.model_selection import train_test_split

#run functions
# %cd scripts # if needed
%run -i scripts/pandas_startup.py
import scripts.functions as functions # because it's in scripts/functions.py

### Get data

**Option 1**: download data from https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783.

In [29]:
df=pd.read_csv("data/Police_Department_Incident_Reports__2018_to_Present.csv")

**Option 2**: use API

In [27]:
limit = "10"
functions.get_data(limit = limit)

Requesting 10 rows from API...
API request status: <Response [200]>
Got the data and put it in data . It took 0.0 seconds.


,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,report_type_code,report_type_description,incident_code,incident_category,incident_subcategory,incident_description,resolution,police_district,filed_online,cad_number
0,2021-01-30T09:43:00.000,2021-01-30T00:00:00.000,2022-04-05 09:43:00,2021,Saturday,2021-01-30T09:43:00.000,100069707045,1000697,210038063,VS,Vehicle Supplement,7045,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Truck",Open or Active,Out of SF,NaN,NaN
1,2021-01-29T11:20:00.000,2021-01-29T00:00:00.000,2022-04-05 11:20:00,2021,Friday,2021-01-29T12:19:00.000,100118006244,1001180,216011027,II,Coplogic Initial,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Ingleside,1.0,NaN
2,2021-01-17T11:59:00.000,2021-01-17T00:00:00.000,2022-04-05 11:59:00,2021,Sunday,2021-01-18T04:39:00.000,100189128150,1001891,216012075,II,Coplogic Initial,28150,Malicious Mischief,Vandalism,"Malicious Mischief, Vandalism to Property",Open or Active,Southern,1.0,NaN
3,2021-02-03T05:31:00.000,2021-02-03T00:00:00.000,2022-04-05 05:31:00,2021,Wednesday,2021-02-03T05:31:00.000,100197219400,1001972,210075554,II,Initial,19400,Disorderly Conduct,Other,Gang Related (secondary only),Cite or Arrest Adult,Out of SF,NaN,210340313.0
4,2021-02-03T12:25:00.000,2021-02-03T00:00:00.000,2022-04-05 12:25:00,2021,Wednesday,2021-02-03T13:40:00.000,100226528150,1002265,216012514,II,Coplogic Initial,28150,Malicious Mischief,Vandalism,"Malicious Mischief, Vandalism to Property",Open or Active,Richmond,1.0,NaN
5,2021-02-04T14:57:00.000,2021-02-04T00:00:00.000,2022-04-05 14:57:00,2021,Thursday,2021-02-04T14:57:00.000,100241007045,1002410,210041903,VS,Vehicle Supplement,7045,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Truck",Open or Active,Out of SF,NaN,NaN
6,2021-01-16T21:00:00.000,2021-01-16T00:00:00.000,2022-04-05 21:00:00,2021,Saturday,2021-01-17T12:06:00.000,100257971000,1002579,216012768,II,Coplogic Initial,71000,Lost Property,Lost Property,Lost Property,Open or Active,Central,1.0,NaN
7,2021-02-05T13:50:00.000,2021-02-05T00:00:00.000,2022-04-05 13:50:00,2021,Friday,2021-02-05T13:50:00.000,100269507041,1002695,210049452,VS,Vehicle Supplement,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,Out of SF,NaN,NaN
8,2021-02-06T13:30:00.000,2021-02-06T00:00:00.000,2022-04-05 13:30:00,2021,Saturday,2021-02-06T19:44:00.000,100430106374,1004301,216015154,II,Coplogic Initial,6374,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,Taraval,1.0,NaN
9,2021-02-09T15:00:00.000,2021-02-09T00:00:00.000,2022-04-05 15:00:00,2021,Tuesday,2021-02-10T10:15:00.000,100469207020,1004692,210091629,VS,Vehicle Supplement,7020,Motor Vehicle Theft,Motor Vehicle Theft,"Vehicle, Stolen & Recovered",Open or Active,Ingleside,NaN,210410059.0


In [19]:
print(len(df))

574951


In [20]:
df.groupby(['Incident Category']).size().sort_values()
df_property_assault = df[df['Incident Category'].isin(["Stolen Property", "Robbery", "Burglary", "Larceny Theft", "Assault"])]

In [21]:
df_property_assault.head()
df_property_assault.isna().sum()
df_property_assault['Incident Date'].sort_values()
daily_incidents = df_property_assault.groupby(['Incident Date']).size().reset_index()

In [22]:
daily_incidents['count'] = daily_incidents.iloc[:,1]
daily_incidents = daily_incidents[['Incident Date', 'count']]
daily_incidents.head()

,Incident Date,count
0,2018/01/01,214
1,2018/01/02,172
2,2018/01/03,203
3,2018/01/04,224
4,2018/01/05,198


In [23]:
# Prophet requires these column names: ds and y.
daily_incidents['ds'] = daily_incidents['Incident Date']
daily_incidents['y'] = daily_incidents['count']
df_prophet = daily_incidents[["ds", "y"]]
df_prophet.head(10).sort_values(by="ds")

,ds,y
0,2018/01/01,214
1,2018/01/02,172
2,2018/01/03,203
3,2018/01/04,224
4,2018/01/05,198
5,2018/01/06,217
6,2018/01/07,190
7,2018/01/08,185
8,2018/01/09,199
9,2018/01/10,172


In [28]:
df_prophet.to_csv("data/daily_incidents_clean.csv")